In [ ]:
import pymskt as mskt
from NSM.mesh.interpolate import interpolate_points, interpolate_mesh, interpolate_common
from NSM.datasets.sdf_dataset import get_pts_center_and_scale
from NSM.mesh import create_mesh
from NSM.models import TriplanarDecoder
import pyvista as pv
import vtk
import open3d as o3d

from itkwidgets import view
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import json
from scipy.spatial import cKDTree
import torch
import torch.nn.functional as F
import pandas as pd

os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Define model parameters to use
TRAIN_DIR = "run_v30" # TO DO: Choose training directory containing model ckpt and latent codes
cwd = os.getcwd()
if TRAIN_DIR not in cwd:
    os.chdir(cwd + '/' + TRAIN_DIR)
print("\033[92mWorking directory set to: ", os.getcwd())
CKPT = '3000' # TO DO: Choose the ckpt value you want to analyze results for
LC_PATH = 'latent_codes' + '/' + CKPT + '.pth'
MODEL_PATH = 'model' + '/' + CKPT + '.pth'

In [ ]:
# Define functions

class NumpyTransform:
    def __init__(self, matrix):
        self.matrix = matrix
    def GetMatrix(self):
        vtk_mat = vtk.vtkMatrix4x4()
        for i in range(4):
            for j in range(4):
                vtk_mat.SetElement(i, j, self.matrix[i, j])
        return vtk_mat

def load_config(config_path='model_params_config.json'):
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        print(f"\033[92mLoaded config from {config_path}\033[0m")
        return config
    except FileNotFoundError:
        raise FileNotFoundError(f"Error: model_params_config.json not found at {config_path}")

def load_model_and_latents(MODEL_PATH, LC_PATH, config, device):
    # Load model
    triplane_args = {
        'latent_dim': config['latent_size'],
        'n_objects': config['objects_per_decoder'],
        'conv_hidden_dims': config['conv_hidden_dims'],
        'conv_deep_image_size': config['conv_deep_image_size'],
        'conv_norm': config['conv_norm'], 
        'conv_norm_type': config['conv_norm_type'],
        'conv_start_with_mlp': config['conv_start_with_mlp'],
        'sdf_latent_size': config['sdf_latent_size'],
        'sdf_hidden_dims': config['sdf_hidden_dims'],
        'sdf_weight_norm': config['weight_norm'],
        'sdf_final_activation': config['final_activation'],
        'sdf_activation': config['activation'],
        'sdf_dropout_prob': config['dropout_prob'],
        'sum_sdf_features': config['sum_conv_output_features'],
        'conv_pred_sdf': config['conv_pred_sdf'],
    }
    model = TriplanarDecoder(**triplane_args)
    model_ckpt = torch.load(MODEL_PATH, map_location=device)
    model.load_state_dict(model_ckpt['model'])
    model.to(device)
    model.eval()
    # Load latents
    latent_ckpt = torch.load(LC_PATH, map_location=device)
    latent_codes = latent_ckpt['latent_codes']['weight'].detach().cpu()
    return model, latent_ckpt, latent_codes

class MeshWrapper:
    def __init__(self, mesh):
        self.mesh = mesh
    def __getattr__(self, name):
        return getattr(self.mesh, name)
    @property
    def scalar_names(self):
        return []
    @property
    def point_coords(self):
        return self.mesh.points
    @point_coords.setter
    def point_coords(self, value):
        self.mesh.points = value

def normalize_latent(v):
    v = np.array(v, dtype=np.float64).reshape(-1)
    norm = np.linalg.norm(v)
    if not np.isfinite(norm) or norm < 1e-9:
        raise ValueError(f"Invalid latent vector: norm={norm}")
    return v / norm

def load_mrk_json(path):
    with open(path, 'r') as f:
        data = json.load(f)
    if "markups" not in data or len(data["markups"]) == 0:
        raise ValueError(f"No 'markups' found in {path}")
    markups = data["markups"][0]
    points = []
    labels = []
    for cp in markups.get("controlPoints", []):
        pos = cp.get("position", None)
        if pos is not None:
            points.append(pos)
            labels.append(cp.get("label", None))
    points = np.array(points, dtype=np.float32)
    return points, labels

def vtk_to_o3d(vtk_file):
    # Load the VTK file using PyVista
    vtk_mesh = pv.read(vtk_file)
    # Extract vertices and faces from the VTK mesh
    vertices = vtk_mesh.points
    faces = vtk_mesh.faces.reshape((-1, 4))[:, 1:]  # Reshape faces to get rid of the first number
    # Create Open3D TriangleMesh from the extracted vertices and faces
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(faces)
    # Optionally, check if normals exist in the VTK file
    if vtk_mesh.point_data.get("Normals") is not None:
        mesh.vertex_normals = o3d.utility.Vector3dVector(vtk_mesh.point_data["Normals"])
    return mesh

def project_landmarks_to_surface(landmarks, target_mesh):
    mesh_points = np.asarray(target_mesh.points)
    kdtree = cKDTree(mesh_points)
    _, idx = kdtree.query(landmarks)
    projected = mesh_points[idx]
    return projected

def visualize_landmark_interpolation_external(original_mesh, mesh, original_pts, interp_pts, labels=None):
    # Unwrap MeshWrapper if needed
    if hasattr(mesh, "mesh"):
        mesh = mesh.mesh
    # Ensure both meshes are PyVista PolyData
    mesh_pv = mesh if isinstance(mesh, pv.PolyData) else mesh.extract_geometry()
    original_mesh_pv = original_mesh if isinstance(original_mesh, pv.PolyData) else original_mesh.extract_geometry()
    # Compute normals for smoother rendering
    mesh_pv = mesh_pv.compute_normals(cell_normals=False, point_normals=True, inplace=False)
    original_mesh_pv = original_mesh_pv.compute_normals(cell_normals=False, point_normals=True, inplace=False)
    # Landmarks as PolyData
    orig = pv.PolyData(original_pts)
    interp = pv.PolyData(interp_pts)
    # Lines connecting original → interpolated
    lines = [pv.Line(original_pts[i], interp_pts[i]) for i in range(len(original_pts))]
    lines_polydata = lines[0]
    for line in lines[1:]:
        lines_polydata = lines_polydata.merge(line)
    # ---- Interactive window ----
    plotter = pv.Plotter(window_size=[1024, 768], notebook=False, off_screen=False)
    plotter.add_text("Landmark Interpolation", font_size=12)
    # Add meshes and landmarks
    plotter.add_mesh(original_mesh_pv, color="lightgray", opacity=0.4, smooth_shading=True, show_edges=False)
    plotter.add_mesh(mesh_pv, color="blue", opacity=0.4, smooth_shading=True, show_edges=False)
    plotter.add_mesh(orig, color="red", point_size=14, render_points_as_spheres=True)
    plotter.add_mesh(interp, color="green", point_size=14, render_points_as_spheres=True)
    plotter.add_mesh(lines_polydata, color="orange", line_width=2)
    # Optional labels
    if labels is not None:
        for pt, label in zip(original_pts, labels):
            plotter.add_point_labels([pt], [label], font_size=10, text_color="black")
    # Show in a standalone GUI window
    plotter.show_axes()
    plotter.show_grid()
    plotter.show(interactive=True, auto_close=True)

def visualize_landmarks_on_mesh(original_mesh, original_pts):
    # Ensure both meshes are PyVista PolyData
    original_mesh_pv = original_mesh if isinstance(original_mesh, pv.PolyData) else original_mesh.extract_geometry()
    # Compute normals for smoother rendering
    original_mesh_pv = original_mesh_pv.compute_normals(cell_normals=False, point_normals=True, inplace=False)
    # Landmarks as PolyData
    orig = pv.PolyData(original_pts)
    # ---- Interactive window ----
    plotter = pv.Plotter(window_size=[1024, 768], notebook=False, off_screen=False)
    plotter.add_text("Landmark Interpolation", font_size=12)
    # Add meshes and landmarks
    plotter.add_mesh(original_mesh_pv, color="lightgray", opacity=0.4, smooth_shading=True, show_edges=False)
    plotter.add_mesh(orig, color="red", point_size=14, render_points_as_spheres=True)
    plotter.show_axes()
    plotter.show_grid()
    # Show in a standalone GUI window
    plotter.show(interactive=True, auto_close=True)

def normalize_to_unit_sphere(landmarks, reference_points=None):
    if reference_points is None:
        reference_points = landmarks
    center = np.mean(reference_points, axis=0)
    shifted = landmarks - center
    scale = np.max(np.linalg.norm(reference_points - center, axis=1))
    normalized = shifted / scale
    return normalized, center, scale

def normalize_mesh_to_unit_sphere(mesh, reference_points=None):
    # Extract vertices
    if isinstance(mesh, pv.PolyData):
        vertices = mesh.points.copy()
    else:  # assume numpy array of shape (N, 3)
        vertices = mesh.copy()
    if reference_points is None:
        reference_points = vertices
    # Compute center and scale
    center = np.mean(reference_points, axis=0)
    shifted = vertices - center
    scale = np.max(np.linalg.norm(reference_points - center, axis=1))
    normalized_vertices = shifted / scale
    # If PyVista mesh, return new mesh
    if isinstance(mesh, pv.PolyData):
        normalized_mesh = mesh.copy()
        normalized_mesh.points = normalized_vertices
        return normalized_mesh, center, scale
    else:
        return normalized_vertices, center, scale
    
def o3d_to_pv(mesh_o3d):
    # Get vertices as numpy array
    vertices = np.asarray(mesh_o3d.vertices)
    # Get faces (Open3D uses shape (n_faces, 3))
    faces = np.asarray(mesh_o3d.triangles)
    # PyVista expects faces in a flattened format: [3, v0, v1, v2, 3, ...]
    faces_flat = np.hstack([np.full((faces.shape[0], 1), 3), faces]).flatten()
    
    # Create PyVista mesh
    mesh_pv = pv.PolyData(vertices, faces_flat)
    return mesh_pv

def save_mrk_json(points, labels, output_filename):
    control_points = []
    for i, (pt, label) in enumerate(zip(points, labels), start=1):
        # Ensure pt is a list or numpy array of coordinates
        if isinstance(pt, np.ndarray):
            pt = pt.tolist()  # If pt is a numpy array, convert to a list
        elif isinstance(pt, list):
            pass  # pt is already a list, no conversion needed
        else:
            raise ValueError(f"Expected pt to be a list or numpy array, got {type(pt)}")
        
        cp = {
            "id": str(i),
            "label": label,
            "description": "",
            "associatedNodeID": "",
            "position": pt,  # No need to call .tolist() here if it's already a list
            "orientation": [-1.0, -0.0, -0.0, -0.0, -1.0, -0.0, 0.0, 0.0, 1.0],
            "selected": True,
            "locked": True,  # Lock the position so it can't be moved in Slicer
            "lockedPosition": True,  # Optional: If you want explicit "lockedPosition" field
            "visibility": True,
            "positionStatus": "defined"  # Ensure the position is defined and not undefined
        }
        control_points.append(cp)

    markups_data = {
        "@schema": "https://raw.githubusercontent.com/slicer/slicer/master/Modules/Loadable/Markups/Resources/Schema/markups-schema-v1.0.3.json#",
        "markups": [
            {
                "type": "Fiducial",
                "coordinateSystem": "LPS",
                "coordinateUnits": "mm",
                "locked": False,  # Keep the overall markup locked status as False for editing purposes
                "fixedNumberOfControlPoints": False,
                "labelFormat": "%N-%d",
                "lastUsedControlPointNumber": len(control_points),
                "controlPoints": control_points,
                "measurements": [],
                "display": {
                    "visibility": False,
                    "opacity": 1.0,
                    "color": [0.4, 1.0, 1.0],
                    "selectedColor": [1.0, 0.5000076295109483, 0.5000076295109483],
                    "activeColor": [0.4, 1.0, 0.0],
                    "propertiesLabelVisibility": False,
                    "pointLabelsVisibility": False,
                    "textScale": 3.0,
                    "glyphType": "Sphere3D",
                    "glyphScale": 3.0,
                    "glyphSize": 5.0,
                    "useGlyphScale": True,
                    "sliceProjection": False,
                    "sliceProjectionUseFiducialColor": True,
                    "sliceProjectionOutlinedBehindSlicePlane": False,
                    "sliceProjectionColor": [1.0, 1.0, 1.0],
                    "sliceProjectionOpacity": 0.6,
                    "lineThickness": 0.2,
                    "lineColorFadingStart": 1.0,
                    "lineColorFadingEnd": 10.0,
                    "lineColorFadingSaturation": 1.0,
                    "lineColorFadingHueOffset": 0.0,
                    "handlesInteractive": False,
                    "translationHandleVisibility": True,
                    "rotationHandleVisibility": True,
                    "scaleHandleVisibility": False,
                    "interactionHandleScale": 3.0,
                    "snapMode": "toVisibleSurface"}}]}

    # Save to JSON file
    with open(output_filename, 'w') as f:
        json.dump(markups_data, f, indent=4)
    print(f"Saved to {output_filename}")

In [ ]:
# Load config
config = load_config()
device = config.get("device", "cuda:0")
train_paths = config['list_mesh_paths']
all_vtk_files = [os.path.basename(f) for f in train_paths]

# Load model and latent codes
model, latent_ckpt, latent_codes = load_model_and_latents(MODEL_PATH, LC_PATH, config, device)

# Mesh creation params
recon_grid_origin = 1.0
n_pts_per_axis = 128
voxel_origin = (-recon_grid_origin, -recon_grid_origin, -recon_grid_origin)
voxel_size = (recon_grid_origin * 2) / (n_pts_per_axis - 1)
offset = np.array([0.0, 0.0, 0.0])
scale = 1.0
icp_transform = NumpyTransform(np.eye(4))
objects = 1

In [ ]:
# Find the specimen closest to the median, and use it and it's landmarks as a template

# Move latent codes to device
latent_codes = latent_codes.to(device)

# Compute median latent
median_latent = torch.median(latent_codes, dim=0).values  # shape: (latent_dim,)

# Normalize both sets of vectors to unit length
latent_norm = F.normalize(latent_codes, p=2, dim=1)
median_latent_norm = F.normalize(median_latent.unsqueeze(0), p=2, dim=1)

# Cosine similarity and distance
cosine_similarities = torch.mm(latent_norm, median_latent_norm.T).squeeze()
cosine_distances = 1 - cosine_similarities

# Closest latent by cosine
med_idx = torch.argmin(cosine_distances).item()

print(f"Closest latent to median (cosine): {all_vtk_files[med_idx]}")

med_fn = "../vertebrae_meshes/" + all_vtk_files[med_idx]
med_lm_fn = '../alignedLMs/' + os.path.splitext(all_vtk_files[med_idx])[0] + '.mrk.json'
print("Associated landmark file: ", med_lm_fn, "\033[0m\n")

med_mesh = vtk_to_o3d(med_fn)
med_mesh = o3d_to_pv(med_mesh)
med_lms, med_labels = load_mrk_json(med_lm_fn)
median_mesh, mesh_center, mesh_scale = normalize_mesh_to_unit_sphere(med_mesh)
normalized_lms = (med_lms - mesh_center) / mesh_scale
median_lms = project_landmarks_to_surface(normalized_lms, median_mesh.mesh if hasattr(median_mesh, "mesh") else median_mesh)
visualize_landmarks_on_mesh(median_mesh, median_lms)

In [ ]:
import shutil

In [ ]:
# Interpolate points from median specimen to random specimens

# Number of meshes to sample
NUM = 5
# create random indices
random_indices = np.random.randint(0, len(latent_codes), NUM)

# ---- Main interpolation ----
interpolated_points = []
meshes = []

# Loop through randomly selected latent codes for interpolation
for i, idx in enumerate(random_indices):
    print(f'\n\033[92m{i}: {all_vtk_files[idx]}\033[0m')
    # Set out filename for landmarks
    out_lm_fn = '../interpedLMs/' + 'interped_' + os.path.splitext(all_vtk_files[idx])[0] + '.mrk.json'
    out_mesh_fn = '../interpedMeshes/' + 'interped_' + os.path.splitext(all_vtk_files[idx])[0] + '.vtk'
    # Load latent codes by index
    latent = latent_codes[idx]
    # Create mesh from latent
    mesh_out = create_mesh(
            decoder=model, latent_vector=latent, n_pts_per_axis=n_pts_per_axis,
            voxel_origin=voxel_origin, voxel_size=voxel_size, path_original_mesh=None,
            offset=offset, scale=scale, icp_transform=icp_transform,
            objects=objects, verbose=False, device=device
        )
    mesh_out = mesh_out[0] if isinstance(mesh_out, list) else mesh_out
    mesh_out.resample_surface(clusters=20_000)

    # Normalize latent vectors
    latent1 = normalize_latent(median_latent.cpu().numpy())
    latent2 = normalize_latent(latent.cpu().numpy())

    # --- Interpolate landmarks (these are now in normalized space) ---
    interp_pts = interpolate_points(
        model=model,
        latent1=latent1,
        latent2=latent2,
        points1=median_lms,
        n_steps=100,
    )
    
    normalized_mesh, mesh_center, mesh_scale = normalize_mesh_to_unit_sphere(mesh_out)
    normalized_lms = (interp_pts - mesh_center) / mesh_scale
    projected_lms = project_landmarks_to_surface(interp_pts, normalized_mesh.mesh if hasattr(normalized_mesh, "mesh") else normalized_mesh)
    
    meshes.append(normalized_mesh)
    interpolated_points.append(projected_lms)

    # Optional: Save interpolated landmarks and generated meshes
    save_mrk_json(projected_lms, med_labels, out_lm_fn)
    orig_lm_fn = os.path.splitext(all_vtk_files[idx])[0] + '.mrk.json'
    shutil.copy('../alignedLMs/' + orig_lm_fn, '../interpedLMs/' + orig_lm_fn)
    print(f"Copied original landmark file to {'../interpedLMs/' + orig_lm_fn} for comparison with interpolated landmarks.")
    normalized_mesh.save(out_mesh_fn)

In [ ]:
# Visualize interpolated landmarks and generated mesh in interactive viewer

# Pick index to inspect
idx_to_show = 2 # change index as needed
pv.set_jupyter_backend(None)  # disable notebook mode completely

visualize_landmark_interpolation_external(
    median_mesh, # original mesh
    meshes[idx_to_show], # generated mesh
    median_lms,  # original landmarks
    interpolated_points[idx_to_show],  # interpolated landmarks
    med_labels,  # labels
)